# Collateral Analysis

This notebook analyzes ZKC collateral locked for fulfilled requests on January 19th, 2026.

It compares:
- Current collateral amounts
- Hypothetical collateral if it was set to lock_fee * 100
- Hypothetical collateral if it was set to lock_fee * 1000

Using a hardcoded ZKC/ETH exchange rate of 0.1.

In [15]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import time

# API configuration
API_BASE_URL = "https://d2mdvlnmyov1e1.cloudfront.net"

# Hardcoded prices
ZKC_USD_PRICE = 0.1  # 1 ZKC = $0.1 USD
ETH_USD_PRICE = 3000.0  # 1 ETH = $3000 USD

print(f"API Base URL: {API_BASE_URL}")
print(f"ZKC/USD Price: ${ZKC_USD_PRICE} (1 ZKC = ${ZKC_USD_PRICE} USD)")
print(f"ETH/USD Price: ${ETH_USD_PRICE:,.0f} (1 ETH = ${ETH_USD_PRICE:,.0f} USD)")

API Base URL: https://d2mdvlnmyov1e1.cloudfront.net
ZKC/USD Price: $0.1 (1 ZKC = $0.1 USD)
ETH/USD Price: $3,000 (1 ETH = $3,000 USD)


## Fetch Fulfilled Requests

Fetch all fulfilled requests from January 19th, 2026.

In [5]:
def wei_to_eth(wei_str):
    """Convert wei (as string) to ETH (float)."""
    try:
        return float(wei_str) / 1e18
    except (ValueError, TypeError):
        return 0.0


def wei_to_zkc(wei_str):
    """Convert wei (as string) to ZKC (float)."""
    try:
        return float(wei_str) / 1e18
    except (ValueError, TypeError):
        return 0.0


def fetch_fulfilled_requests(target_date_str='2026-01-19', limit=500):
    """
    Fetch all fulfilled requests from a specific date.
    
    Args:
        target_date_str: Target date in YYYY-MM-DD format
        limit: Maximum results per page (max 500)
    
    Returns:
        pandas DataFrame with fulfilled requests
    """
    all_requests = []
    cursor = None
    
    # Parse target date
    target_date = pd.to_datetime(target_date_str).date()
    day_before = target_date - pd.Timedelta(days=1)
    
    print(f"Fetching fulfilled requests from {target_date_str}...")
    
    while True:
        params = {
            'limit': limit
        }
        
        if cursor:
            params['cursor'] = cursor
        
        try:
            response = requests.get(
                f"{API_BASE_URL}/v1/market/requests",
                params=params,
                timeout=30
            )
            response.raise_for_status()
            data = response.json()
            
            requests_data = data.get('data', [])
            
            if not requests_data:
                break
            
            # Track dates in this batch for logging
            dates_seen = []
            # Track if we've seen any requests from before the target date
            seen_older_date = False
            
            # Filter for fulfilled requests on target date
            for req in requests_data:
                # Check date to see if we've gone too far back
                fulfilled_at_iso = req.get('fulfilled_at_iso')
                created_at_iso = req.get('created_at_iso')
                
                # Use fulfilled_at_iso if available, otherwise created_at_iso as fallback
                date_iso = fulfilled_at_iso or created_at_iso
                if date_iso:
                    req_date = pd.to_datetime(date_iso).date()
                    dates_seen.append(req_date)
                    
                    # If we see a request from before the target date, we've gone too far
                    if req_date < target_date:
                        seen_older_date = True
                    
                    # Add fulfilled requests from target date
                    if req.get('request_status') == 'fulfilled' and fulfilled_at_iso:
                        fulfilled_date = pd.to_datetime(fulfilled_at_iso).date()
                        if fulfilled_date == target_date:
                            all_requests.append(req)
            
            # Log the date range we're currently processing
            if dates_seen:
                min_date = min(dates_seen)
                max_date = max(dates_seen)
                print(f"  Processing batch: dates from {min_date} to {max_date} ({len(requests_data)} requests, {len(all_requests)} matching so far)")
            
            # If we've seen requests from before the target date, we've processed everything
            if seen_older_date:
                print(f"Reached requests from before {target_date_str}, stopping pagination")
                break
            
            # Check for more pages
            if not data.get('has_more', False):
                break
            
            cursor = data.get('next_cursor')
            if not cursor:
                break
            
            # Small delay to be nice to the API
            time.sleep(0.1)
                
        except Exception as e:
            print(f"Error fetching requests: {e}")
            break
    
    print(f"Found {len(all_requests)} fulfilled requests from {target_date_str}")
    
    if not all_requests:
        return pd.DataFrame()
    
    # Convert to DataFrame
    df = pd.DataFrame(all_requests)
    
    # Extract and convert relevant fields
    df['lock_collateral_zkc'] = df['lock_collateral'].apply(wei_to_zkc)
    df['lock_price_eth'] = df['lock_price'].apply(wei_to_eth)
    
    # Parse timestamps
    if 'fulfilled_at_iso' in df.columns:
        df['fulfilled_at'] = pd.to_datetime(df['fulfilled_at_iso'])
    
    return df

## Collect Data

In [6]:
# Fetch fulfilled requests from January 19th, 2026
fulfilled_requests = fetch_fulfilled_requests(target_date_str='2026-01-19')

if not fulfilled_requests.empty:
    print(f"\nTotal fulfilled requests: {len(fulfilled_requests)}")
    print(f"\nSample data:")
    print(fulfilled_requests[['request_id', 'lock_collateral_zkc', 'lock_price_eth', 'fulfilled_at']].head(10))
else:
    print("No fulfilled requests found for the specified date.")

Fetching fulfilled requests from 2026-01-19...
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-22 to 2026-01-22 (500 requests, 0 matching so far)
  Processing batch: dates from 2026-01-21 to 2026-01-

## Detailed Request Breakdown

Display all requests with their collateral values for inspection.

In [37]:
if not fulfilled_requests.empty:
    # Calculate alternative collateral for each request
    # If collateral = lock_price_eth * 10, convert to ZKC: (lock_price_eth * 10) / (ETH_USD_PRICE * ZKC_USD_PRICE)
    # Since lock_price_eth is in ETH, we need: lock_price_eth * 10 / (ETH_USD_PRICE / ZKC_USD_PRICE)
    # Actually, simpler: lock_price_eth * 10 in USD = lock_price_eth * 10 * ETH_USD_PRICE
    # Then convert USD to ZKC: (lock_price_eth * 10 * ETH_USD_PRICE) / ZKC_USD_PRICE
    fulfilled_requests['alt_collateral_10x'] = (fulfilled_requests['lock_price_eth'] * 10 * ETH_USD_PRICE) / ZKC_USD_PRICE
    fulfilled_requests['alt_collateral_100x'] = (fulfilled_requests['lock_price_eth'] * 100 * ETH_USD_PRICE) / ZKC_USD_PRICE
    fulfilled_requests['alt_collateral_1000x'] = (fulfilled_requests['lock_price_eth'] * 1000 * ETH_USD_PRICE) / ZKC_USD_PRICE
    
    # Calculate USD values
    # Current collateral in USD: ZKC * ZKC_USD_PRICE (direct conversion)
    fulfilled_requests['lock_collateral_usd'] = fulfilled_requests['lock_collateral_zkc'] * ZKC_USD_PRICE
    fulfilled_requests['lock_price_usd'] = fulfilled_requests['lock_price_eth'] * ETH_USD_PRICE
    fulfilled_requests['alt_collateral_10x_usd'] = fulfilled_requests['alt_collateral_10x'] * ZKC_USD_PRICE
    fulfilled_requests['alt_collateral_100x_usd'] = fulfilled_requests['alt_collateral_100x'] * ZKC_USD_PRICE
    fulfilled_requests['alt_collateral_1000x_usd'] = fulfilled_requests['alt_collateral_1000x'] * ZKC_USD_PRICE
    
    # Create display table
    display_df = fulfilled_requests[[
        'request_id',
        'lock_collateral_zkc',
        'lock_collateral_usd',
        'lock_price_eth',
        'lock_price_usd',
        'alt_collateral_10x',
        'alt_collateral_10x_usd',
        'alt_collateral_100x',
        'alt_collateral_100x_usd',
        'alt_collateral_1000x',
        'alt_collateral_1000x_usd'
    ]].copy()
    
    # Format for display - show full request ID
    display_df['request_id'] = display_df['request_id'].apply(str)
    display_df['lock_collateral_zkc'] = display_df['lock_collateral_zkc'].apply(lambda x: f"{x:.2f}")
    display_df['lock_collateral_usd'] = display_df['lock_collateral_usd'].apply(lambda x: f"${x:.2f}")
    display_df['lock_price_eth'] = display_df['lock_price_eth'].apply(lambda x: f"{x:.10f}")
    display_df['lock_price_usd'] = display_df['lock_price_usd'].apply(lambda x: f"${x:.6f}")
    display_df['alt_collateral_10x'] = display_df['alt_collateral_10x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_10x_usd'] = display_df['alt_collateral_10x_usd'].apply(lambda x: f"${x:.2f}")
    display_df['alt_collateral_100x'] = display_df['alt_collateral_100x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_100x_usd'] = display_df['alt_collateral_100x_usd'].apply(lambda x: f"${x:.2f}")
    display_df['alt_collateral_1000x'] = display_df['alt_collateral_1000x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_1000x_usd'] = display_df['alt_collateral_1000x_usd'].apply(lambda x: f"${x:.2f}")
    
    # Rename columns for clarity
    display_df.columns = [
        'Request ID',
        'Current Collateral (ZKC)',
        'Current Collateral (USD)',
        'Lock Price (ETH)',
        'Lock Price (USD)',
        'Alt Collateral 10x (ZKC)',
        'Alt Collateral 10x (USD)',
        'Alt Collateral 100x (ZKC)',
        'Alt Collateral 100x (USD)',
        'Alt Collateral 1000x (ZKC)',
        'Alt Collateral 1000x (USD)'
    ]
    
    print(f"Detailed breakdown of {len(display_df)} requests:")
    print("=" * 180)
    
    # Display all rows with full column width
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 100)
    pd.set_option('display.max_columns', None)
    
    print(display_df.to_string(index=False))
    
    # Reset display options
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')
else:
    print("No data available for display.")

Detailed breakdown of 6496 requests:
                                         Request ID Current Collateral (ZKC) Current Collateral (USD) Lock Price (ETH) Lock Price (USD) Alt Collateral 10x (ZKC) Alt Collateral 10x (USD) Alt Collateral 100x (ZKC) Alt Collateral 100x (USD) Alt Collateral 1000x (ZKC) Alt Collateral 1000x (USD)
 0x7276fa3c23c1783bdcb97258717f8f32b75919c97235c24e                    15.00                    $1.50     0.0000062906        $0.018872                     1.89                    $0.19                     18.87                     $1.89                     188.72                     $18.87
 0x7276fa3c23c1783bdcb97258717f8f32b75919c9fd1da055                    15.00                    $1.50     0.0000189039        $0.056712                     5.67                    $0.57                     56.71                     $5.67                     567.12                     $56.71
 0xc197ebe12c7bcf1d9f3b415342bdbc795425335cc668cd8c                    15.00           

## Calculate Collateral Totals

In [38]:
if not fulfilled_requests.empty:
    # Current collateral (sum of all lock_collateral)
    current_total_zkc = fulfilled_requests['lock_collateral_zkc'].sum()
    
    # Alternative collateral calculations
    # If collateral = lock_price_eth * 10, convert to ZKC:
    if 'alt_collateral_10x' not in fulfilled_requests.columns:
        fulfilled_requests['alt_collateral_10x'] = (fulfilled_requests['lock_price_eth'] * 10 * ETH_USD_PRICE) / ZKC_USD_PRICE
    
    # If collateral = lock_price_eth * 100, convert to ZKC:
    if 'alt_collateral_100x' not in fulfilled_requests.columns:
        fulfilled_requests['alt_collateral_100x'] = (fulfilled_requests['lock_price_eth'] * 100 * ETH_USD_PRICE) / ZKC_USD_PRICE
    
    # If collateral = lock_price_eth * 1000, convert to ZKC:
    if 'alt_collateral_1000x' not in fulfilled_requests.columns:
        fulfilled_requests['alt_collateral_1000x'] = (fulfilled_requests['lock_price_eth'] * 1000 * ETH_USD_PRICE) / ZKC_USD_PRICE
    
    alt_total_zkc_10x = fulfilled_requests['alt_collateral_10x'].sum()
    alt_total_zkc_100x = fulfilled_requests['alt_collateral_100x'].sum()
    alt_total_zkc_1000x = fulfilled_requests['alt_collateral_1000x'].sum()
    
    # Calculate USD totals (direct conversion: ZKC * ZKC_USD_PRICE)
    if 'lock_collateral_usd' not in fulfilled_requests.columns:
        fulfilled_requests['lock_collateral_usd'] = fulfilled_requests['lock_collateral_zkc'] * ZKC_USD_PRICE
    
    current_total_usd = fulfilled_requests['lock_collateral_usd'].sum()
    alt_total_usd_10x = alt_total_zkc_10x * ZKC_USD_PRICE
    alt_total_usd_100x = alt_total_zkc_100x * ZKC_USD_PRICE
    alt_total_usd_1000x = alt_total_zkc_1000x * ZKC_USD_PRICE
    
    print("Collateral Analysis Results:")
    print("=" * 80)
    print(f"Total fulfilled requests: {len(fulfilled_requests):,}")
    print(f"\nCurrent total ZKC collateral: {current_total_zkc:,.2f} ZKC (${current_total_usd:,.2f})")
    print(f"Alternative total (lock_fee * 10): {alt_total_zkc_10x:,.2f} ZKC (${alt_total_usd_10x:,.2f})")
    print(f"Alternative total (lock_fee * 100): {alt_total_zkc_100x:,.2f} ZKC (${alt_total_usd_100x:,.2f})")
    print(f"Alternative total (lock_fee * 1000): {alt_total_zkc_1000x:,.2f} ZKC (${alt_total_usd_1000x:,.2f})")
    print("\n" + "=" * 80)
    
    # Calculate differences
    diff_10x = alt_total_zkc_10x - current_total_zkc
    diff_100x = alt_total_zkc_100x - current_total_zkc
    diff_1000x = alt_total_zkc_1000x - current_total_zkc
    diff_10x_usd = alt_total_usd_10x - current_total_usd
    diff_100x_usd = alt_total_usd_100x - current_total_usd
    diff_1000x_usd = alt_total_usd_1000x - current_total_usd
    
    print(f"\nDifference (10x vs current): {diff_10x:,.2f} ZKC (${diff_10x_usd:,.2f}) ({diff_10x/current_total_zkc*100:.1f}%)")
    print(f"Difference (100x vs current): {diff_100x:,.2f} ZKC (${diff_100x_usd:,.2f}) ({diff_100x/current_total_zkc*100:.1f}%)")
    print(f"Difference (1000x vs current): {diff_1000x:,.2f} ZKC (${diff_1000x_usd:,.2f}) ({diff_1000x/current_total_zkc*100:.1f}%)")
else:
    print("No data available for analysis.")

Collateral Analysis Results:
Total fulfilled requests: 6,496

Current total ZKC collateral: 116,440.00 ZKC ($11,644.00)
Alternative total (lock_fee * 10): 61,713.44 ZKC ($6,171.34)
Alternative total (lock_fee * 100): 617,134.36 ZKC ($61,713.44)
Alternative total (lock_fee * 1000): 6,171,343.55 ZKC ($617,134.36)


Difference (10x vs current): -54,726.56 ZKC ($-5,472.66) (-47.0%)
Difference (100x vs current): 500,694.36 ZKC ($50,069.44) (430.0%)
Difference (1000x vs current): 6,054,903.55 ZKC ($605,490.36) (5200.0%)


## Filtered Analysis by Requestor Type

Breakdown by requestor type: OG, Signal, and Citrea.

In [39]:
def create_filtered_table(fulfilled_requests, filter_string, requestor_name):
    # Create a filtered table for a specific requestor type
    filtered = fulfilled_requests[fulfilled_requests['request_id'].str.contains(filter_string, case=False, na=False)].copy()
    
    if filtered.empty:
        print(f"No {requestor_name} requests found.")
        # Return empty DataFrame and empty summary
        empty_summary = pd.DataFrame({
            'Scenario': ['Current Collateral', 'Collateral (lock_fee * 10)', 'Collateral (lock_fee * 100)', 'Collateral (lock_fee * 1000)'],
            'Total ZKC': ['0.00', '0.00', '0.00', '0.00'],
            'Total USD': ['$0.00', '$0.00', '$0.00', '$0.00']
        })
        return filtered, empty_summary
    
    # Calculate alternative collateral if not already done
    if 'alt_collateral_10x' not in filtered.columns:
        filtered['alt_collateral_10x'] = (filtered['lock_price_eth'] * 10 * ETH_USD_PRICE) / ZKC_USD_PRICE
    if 'alt_collateral_100x' not in filtered.columns:
        filtered['alt_collateral_100x'] = (filtered['lock_price_eth'] * 100 * ETH_USD_PRICE) / ZKC_USD_PRICE
    if 'alt_collateral_1000x' not in filtered.columns:
        filtered['alt_collateral_1000x'] = (filtered['lock_price_eth'] * 1000 * ETH_USD_PRICE) / ZKC_USD_PRICE
    
    # Calculate USD values
    if 'lock_collateral_usd' not in filtered.columns:
        filtered['lock_collateral_usd'] = filtered['lock_collateral_zkc'] * ZKC_USD_PRICE
    filtered['alt_collateral_10x_usd'] = filtered['alt_collateral_10x'] * ZKC_USD_PRICE
    filtered['alt_collateral_100x_usd'] = filtered['alt_collateral_100x'] * ZKC_USD_PRICE
    filtered['alt_collateral_1000x_usd'] = filtered['alt_collateral_1000x'] * ZKC_USD_PRICE
    
    # Create display table
    display_df = filtered[[
        'request_id',
        'lock_collateral_zkc',
        'lock_collateral_usd',
        'lock_price_eth',
        'lock_price_usd',
        'alt_collateral_10x',
        'alt_collateral_10x_usd',
        'alt_collateral_100x',
        'alt_collateral_100x_usd',
        'alt_collateral_1000x',
        'alt_collateral_1000x_usd'
    ]].copy()
    
    # Format for display
    display_df['request_id'] = display_df['request_id'].apply(str)
    display_df['lock_collateral_zkc'] = display_df['lock_collateral_zkc'].apply(lambda x: f"{x:.2f}")
    display_df['lock_collateral_usd'] = display_df['lock_collateral_usd'].apply(lambda x: f"${x:.2f}")
    display_df['lock_price_eth'] = display_df['lock_price_eth'].apply(lambda x: f"{x:.10f}")
    display_df['lock_price_usd'] = display_df['lock_price_usd'].apply(lambda x: f"${x:.6f}")
    display_df['alt_collateral_10x'] = display_df['alt_collateral_10x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_10x_usd'] = display_df['alt_collateral_10x_usd'].apply(lambda x: f"${x:.2f}")
    display_df['alt_collateral_100x'] = display_df['alt_collateral_100x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_100x_usd'] = display_df['alt_collateral_100x_usd'].apply(lambda x: f"${x:.2f}")
    display_df['alt_collateral_1000x'] = display_df['alt_collateral_1000x'].apply(lambda x: f"{x:.2f}")
    display_df['alt_collateral_1000x_usd'] = display_df['alt_collateral_1000x_usd'].apply(lambda x: f"${x:.2f}")
    
    # Rename columns
    display_df.columns = [
        'Request ID',
        'Current Collateral (ZKC)',
        'Current Collateral (USD)',
        'Lock Price (ETH)',
        'Lock Price (USD)',
        'Alt Collateral 10x (ZKC)',
        'Alt Collateral 10x (USD)',
        'Alt Collateral 100x (ZKC)',
        'Alt Collateral 100x (USD)',
        'Alt Collateral 1000x (ZKC)',
        'Alt Collateral 1000x (USD)'
    ]
    
    # Calculate totals
    total_zkc = filtered['lock_collateral_zkc'].sum()
    total_usd = filtered['lock_collateral_usd'].sum()
    total_10x_zkc = filtered['alt_collateral_10x'].sum()
    total_10x_usd = filtered['alt_collateral_10x_usd'].sum()
    total_100x_zkc = filtered['alt_collateral_100x'].sum()
    total_100x_usd = filtered['alt_collateral_100x_usd'].sum()
    total_1000x_zkc = filtered['alt_collateral_1000x'].sum()
    total_1000x_usd = filtered['alt_collateral_1000x_usd'].sum()
    
    print(f"\n{requestor_name} Requests ({len(filtered)} requests):")
    print("=" * 180)
    
    # Display all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 100)
    pd.set_option('display.max_columns', None)
    
    print(display_df.to_string(index=False))
    
    # Reset display options
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')
    pd.reset_option('display.max_columns')
    
    print(f"\n{requestor_name} Totals:")
    print(f"  Current: {total_zkc:,.2f} ZKC (${total_usd:,.2f})")
    print(f"  10x: {total_10x_zkc:,.2f} ZKC (${total_10x_usd:,.2f})")
    print(f"  100x: {total_100x_zkc:,.2f} ZKC (${total_100x_usd:,.2f})")
    print(f"  1000x: {total_1000x_zkc:,.2f} ZKC (${total_1000x_usd:,.2f})")
    print("=" * 180)
    
    # Create summary DataFrame
    summary_data = {
        'Scenario': [
            'Current Collateral',
            'Collateral (lock_fee * 10)',
            'Collateral (lock_fee * 100)',
            'Collateral (lock_fee * 1000)'
        ],
        'Total ZKC': [
            total_zkc,
            total_10x_zkc,
            total_100x_zkc,
            total_1000x_zkc
        ],
        'Total USD': [
            total_usd,
            total_10x_usd,
            total_100x_usd,
            total_1000x_usd
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df['Total ZKC'] = summary_df['Total ZKC'].apply(lambda x: f"{x:,.2f}")
    summary_df['Total USD'] = summary_df['Total USD'].apply(lambda x: f"${x:,.2f}")
    
    return filtered, summary_df

### OG Requests

In [40]:
if not fulfilled_requests.empty:
    og_requests, og_summary = create_filtered_table(
        fulfilled_requests,
        'e198c6944cae382902a375b0b8673084270a7f8e',
        'OG'
    )


OG Requests (2247 requests):
                                        Request ID Current Collateral (ZKC) Current Collateral (USD) Lock Price (ETH) Lock Price (USD) Alt Collateral 10x (ZKC) Alt Collateral 10x (USD) Alt Collateral 100x (ZKC) Alt Collateral 100x (USD) Alt Collateral 1000x (ZKC) Alt Collateral 1000x (USD)
0xe198c6944cae382902a375b0b8673084270a7f8e611ddb33                    15.00                    $1.50     0.0000002451        $0.000735                     0.07                    $0.01                      0.74                     $0.07                       7.35                      $0.74
0xe198c6944cae382902a375b0b8673084270a7f8edb241573                    15.00                    $1.50     0.0000013209        $0.003963                     0.40                    $0.04                      3.96                     $0.40                      39.63                      $3.96
0xe198c6944cae382902a375b0b8673084270a7f8ef4f1e1d6                    15.00                    $1

### OG Summary

In [41]:
if not fulfilled_requests.empty and 'og_summary' in locals():
    print("OG Summary Comparison:")
    print("=" * 80)
    print(og_summary.to_string(index=False))
    print("=" * 80)

OG Summary Comparison:
                    Scenario    Total ZKC   Total USD
          Current Collateral    33,705.00   $3,370.50
  Collateral (lock_fee * 10)    14,556.36   $1,455.64
 Collateral (lock_fee * 100)   145,563.64  $14,556.36
Collateral (lock_fee * 1000) 1,455,636.36 $145,563.64


### Signal Requests

### Signal Summary

In [42]:
if not fulfilled_requests.empty and 'signal_summary' in locals():
    print("Signal Summary Comparison:")
    print("=" * 80)
    print(signal_summary.to_string(index=False))
    print("=" * 80)

Signal Summary Comparison:
                    Scenario    Total ZKC   Total USD
          Current Collateral    22,400.00   $2,240.00
  Collateral (lock_fee * 10)        10.77       $1.08
 Collateral (lock_fee * 100)   323,030.00  $32,303.00
Collateral (lock_fee * 1000) 3,230,300.00 $323,030.00


In [43]:
if not fulfilled_requests.empty:
    signal_requests, signal_summary = create_filtered_table(
        fulfilled_requests,
        '734df7809c4ef94da037449c287166d114503198',
        'Signal'
    )


Signal Requests (224 requests):
                                         Request ID Current Collateral (ZKC) Current Collateral (USD) Lock Price (ETH) Lock Price (USD) Alt Collateral 10x (ZKC) Alt Collateral 10x (USD) Alt Collateral 100x (ZKC) Alt Collateral 100x (USD) Alt Collateral 1000x (ZKC) Alt Collateral 1000x (USD)
0x1734df7809c4ef94da037449c287166d11450319800066f6c                   100.00                   $10.00     0.0003700000        $1.110000                   111.00                   $11.10                   1110.00                   $111.00                   11100.00                   $1110.00
0x1734df7809c4ef94da037449c287166d11450319800066f6a                   100.00                   $10.00     0.0003466667        $1.040000                   104.00                   $10.40                   1040.00                   $104.00                   10400.00                   $1040.00
0x1734df7809c4ef94da037449c287166d11450319800066f68                   100.00               

### Citrea Summary

In [44]:
if not fulfilled_requests.empty and 'citrea_summary' in locals():
    print("Citrea Summary Comparison:")
    print("=" * 80)
    print(citrea_summary.to_string(index=False))
    print("=" * 80)

Citrea Summary Comparison:
                    Scenario  Total ZKC  Total USD
          Current Collateral   2,850.00    $285.00
  Collateral (lock_fee * 10)       0.64      $0.06
 Collateral (lock_fee * 100)  19,176.52  $1,917.65
Collateral (lock_fee * 1000) 191,765.20 $19,176.52


### Citrea Requests

In [45]:
if not fulfilled_requests.empty:
    citrea_requests, citrea_summary = create_filtered_table(
        fulfilled_requests,
        '7276fa3c23c1783bdcb97258717f8f32b75919c9',
        'Citrea'
    )


Citrea Requests (190 requests):
                                        Request ID Current Collateral (ZKC) Current Collateral (USD) Lock Price (ETH) Lock Price (USD) Alt Collateral 10x (ZKC) Alt Collateral 10x (USD) Alt Collateral 100x (ZKC) Alt Collateral 100x (USD) Alt Collateral 1000x (ZKC) Alt Collateral 1000x (USD)
0x7276fa3c23c1783bdcb97258717f8f32b75919c97235c24e                    15.00                    $1.50     0.0000062906        $0.018872                     1.89                    $0.19                     18.87                     $1.89                     188.72                     $18.87
0x7276fa3c23c1783bdcb97258717f8f32b75919c9fd1da055                    15.00                    $1.50     0.0000189039        $0.056712                     5.67                    $0.57                     56.71                     $5.67                     567.12                     $56.71
0x7276fa3c23c1783bdcb97258717f8f32b75919c9a69b9536                    15.00                   

## Summary Table

In [46]:
if not fulfilled_requests.empty:
    summary_data = {
        'Scenario': [
            'Current Collateral',
            'Collateral (lock_fee * 10)',
            'Collateral (lock_fee * 100)',
            'Collateral (lock_fee * 1000)'
        ],
        'Total ZKC': [
            current_total_zkc,
            alt_total_zkc_10x if 'alt_total_zkc_10x' in locals() else fulfilled_requests['alt_collateral_10x'].sum() if 'alt_collateral_10x' in fulfilled_requests.columns else 0,
            alt_total_zkc_100x if 'alt_total_zkc_100x' in locals() else fulfilled_requests['alt_collateral_100x'].sum() if 'alt_collateral_100x' in fulfilled_requests.columns else 0,
            alt_total_zkc_1000x if 'alt_total_zkc_1000x' in locals() else fulfilled_requests['alt_collateral_1000x'].sum() if 'alt_collateral_1000x' in fulfilled_requests.columns else 0
        ],
        'Total USD': [
            current_total_usd if 'current_total_usd' in locals() else fulfilled_requests['lock_collateral_usd'].sum() if 'lock_collateral_usd' in fulfilled_requests.columns else current_total_zkc * ZKC_USD_PRICE,
            alt_total_usd_10x if 'alt_total_usd_10x' in locals() else (fulfilled_requests['alt_collateral_10x'].sum() * ZKC_USD_PRICE if 'alt_collateral_10x' in fulfilled_requests.columns else 0),
            alt_total_usd_100x if 'alt_total_usd_100x' in locals() else (fulfilled_requests['alt_collateral_100x'].sum() * ZKC_USD_PRICE if 'alt_collateral_100x' in fulfilled_requests.columns else 0),
            alt_total_usd_1000x if 'alt_total_usd_1000x' in locals() else (fulfilled_requests['alt_collateral_1000x'].sum() * ZKC_USD_PRICE if 'alt_collateral_1000x' in fulfilled_requests.columns else 0)
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df['Total ZKC'] = summary_df['Total ZKC'].apply(lambda x: f"{x:,.2f}")
    summary_df['Total USD'] = summary_df['Total USD'].apply(lambda x: f"${x:,.2f}")
    
    print("Summary Comparison:")
    print("=" * 80)
    print(summary_df.to_string(index=False))
    print("=" * 80)

Summary Comparison:
                    Scenario    Total ZKC   Total USD
          Current Collateral   116,440.00  $11,644.00
  Collateral (lock_fee * 10)    61,713.44   $6,171.34
 Collateral (lock_fee * 100)   617,134.36  $61,713.44
Collateral (lock_fee * 1000) 6,171,343.55 $617,134.36
